In [1]:
# Configure tensorflow and GPUs
import tensorflow as tf
gpus = tf.config.list_physical_devices("GPU")
try:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
except:
    pass

## Importing data and modules

In [2]:
# Import all necessary libraries
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import os

In [3]:
# Initialize constants
num_classes = 5
img_rows, img_cols = 48, 48
batch_size = 32

In [4]:
train_data_dir = "train"
validation_data_dir = "validation"

## Image Augmentation

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    shear_range=0.3,
    zoom_range=0.3,
    width_shift_range=0.4,
    height_shift_range=0.4,
    horizontal_flip=True,
    fill_mode="nearest"
)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    color_mode="grayscale",
    target_size=(img_cols, img_rows),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True
)

Found 24256 images belonging to 5 classes.


In [7]:
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    color_mode="grayscale",
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True
)

Found 3006 images belonging to 5 classes.


## Model Creation

In [8]:
model = Sequential()

In [9]:
# Block-1
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',
                 input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',
                 input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Block-2
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Block-3
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Block-4
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Block-5
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#Block-6
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#Block-7
model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0

In [11]:
checkpoint = ModelCheckpoint(
    "EmotionDetectionModel.h5",
    monitor="val_loss",
    mode="min",
    save_best_only=True,
    verbose=1
)

In [12]:
earlystop = EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=3,
    verbose=1,
    restore_best_weights=True
)

In [13]:
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    patience=3,
    verbose=1,
    min_delta=0.0001
)

In [14]:
callbacks = [earlystop, checkpoint, reduce_lr]

In [15]:
model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(lr=0.001),
    metrics=["accuracy"]
)

In [16]:
nb_train_samples = 24176
nb_validation_samples = 3006
epochs=25

In [17]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples//batch_size
)

/usr/lib/python3/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
755/755 [==============================] - 15s 15ms/step - loss: 2.1567 - accuracy: 0.2189 - val_loss: 1.5421 - val_accuracy: 0.3048

Epoch 00001: val_loss improved from inf to 1.54207, saving model to EmotionDetectionModel.h5
Epoch 2/25
755/755 [==============================] - 12s 15ms/step - loss: 1.5953 - accuracy: 0.2835 - val_loss: 1.5310 - val_accuracy: 0.3105

Epoch 00002: val_loss improved from 1.54207 to 1.53100, saving model to EmotionDetectionModel.h5
Epoch 3/25
755/755 [==============================] - 11s 15ms/step - loss: 1.5449 - accuracy: 0.3074 - val_loss: 1.4511 - val_accuracy: 0.3646

Epoch 00003: val_loss improved from 1.53100 to 1.45114, saving model to EmotionDetectionModel.h5
Epoch 4/25
755/755 [==============================] - 11s 15ms/step - loss: 1.5038 - accuracy: 0.3307 - val_loss: 1.4581 - val_accuracy: 0.3901

Epoch 00004: val_loss did not improve from 1.45114
Epoch 5/25
755/755 [==============================] - 11s 15ms/step - loss: 1.4572